<a href="https://colab.research.google.com/github/cswcjt/Deep_Learning/blob/main/tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Nov 30 05:21:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.9.2'

In [3]:
# DNN 
from tensorflow import keras
import tensorflow as tf
# dataset 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Flatten # -> Dense: 모든 노드들이 연결되어 있는 NN
from tensorflow.keras import Sequential

In [4]:
# load CIFAR10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# RGB (3-channel)
# 32*32 -R -> pitxel matrix
# 32*32 -G -> pitxel matrix
# 32*32 -B -> pitxel matrix

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


Model Traing

1. Model architecture
    

In [21]:
model = Sequential([
    Input(shape=(32, 32, 3)), # input layer shape 결정
    Flatten(), # (32,32,3) ---> shape을 (32*32*3, 1)로 바꾸는 작업: (input layer의 node 수)
    Dense(units=1024, activation='relu'),
    Dense(units=512, activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=128, activation='sigmoid'),
    Dense(units=10, activation='softmax') # output layer의 유닛은 클래스의 개수, activation은 소프트맥스                
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 3072)              0         
                                                                 
 dense_8 (Dense)             (None, 1024)              3146752   
                                                                 
 dense_9 (Dense)             (None, 512)               524800    
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dense_11 (Dense)            (None, 128)               32896     
                                                                 
 dense_12 (Dense)            (None, 10)                1290      
                                                                 
Total params: 3,837,066
Trainable params: 3,837,066
No

In [22]:
# dense_1 (Dense)  (None, 1024)   3146752 = (3072 + 1) * 1024
# dense_2 (Dense)  (None, 1024)   3146752 = (1024 + 1) * 1024
# ...

In [23]:
# compile
from tensorflow.keras.optimizers import SGD, Adam # Stochastic, 아담을 부른다

lr = 1e-7
# 모든 데이터가 학습에 사용되면 1 epoch
# epochs: 전체 데이터를 몇번 반복 학습을 수행할건지
# iterations: 실제로 weight update를 한 횟수.
# iterations =  전체 데이터 수 / batch_size.
# ex) 50000 / 128 = 391
# ex) 50000 / 256 = 196
epochs = 40
batch_size = 128

# 파라미터로 momentum 넣을 수 있다
#optimizer_fn = SGD(learning_rate=lr) 
optimizer_fn = SGD(learning_rate=lr, momentum=0.9) 
# optimizer_fn = Adam(learning_rate=lr) 

# 클래스가 여러개인 경우, 즉 target vector가 정수인 경우 적용하는 crossentropy
loss_fn = 'sparse_categorical_crossentropy' 

model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [24]:
# fit 
model.fit(x=X_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(X_test, y_test))

Epoch 1/40
391/391 - 2s - loss: 2.6745 - accuracy: 0.0994 - val_loss: 2.6586 - val_accuracy: 0.1025 - 2s/epoch - 5ms/step
Epoch 2/40
391/391 - 2s - loss: 2.6494 - accuracy: 0.1018 - val_loss: 2.6346 - val_accuracy: 0.1043 - 2s/epoch - 4ms/step
Epoch 3/40
391/391 - 1s - loss: 2.6261 - accuracy: 0.1038 - val_loss: 2.6120 - val_accuracy: 0.1058 - 1s/epoch - 4ms/step
Epoch 4/40
391/391 - 1s - loss: 2.6043 - accuracy: 0.1057 - val_loss: 2.5916 - val_accuracy: 0.1087 - 1s/epoch - 3ms/step
Epoch 5/40
391/391 - 1s - loss: 2.5845 - accuracy: 0.1077 - val_loss: 2.5724 - val_accuracy: 0.1099 - 1s/epoch - 4ms/step
Epoch 6/40
391/391 - 1s - loss: 2.5663 - accuracy: 0.1088 - val_loss: 2.5544 - val_accuracy: 0.1109 - 1s/epoch - 3ms/step
Epoch 7/40
391/391 - 1s - loss: 2.5486 - accuracy: 0.1100 - val_loss: 2.5366 - val_accuracy: 0.1128 - 1s/epoch - 4ms/step
Epoch 8/40
391/391 - 1s - loss: 2.5324 - accuracy: 0.1114 - val_loss: 2.5208 - val_accuracy: 0.1143 - 1s/epoch - 4ms/step
Epoch 9/40
391/391 - 1s 

In [18]:
## 어떤 Hyper-parameter setting에서 최고의 성능이 나왔나요?
# evaluate
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Loss : %.4f, Test Accuracy : %.4f" % (loss, acc))

40/40 [==============================] - 0s 4ms/step - loss: 2.1401 - accuracy: 0.2414
Loss : 2.1401, Test Accuracy : 0.2414
